In [1]:
import cv2
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
import zipfile
import shutil
import os
import random
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.0/612.0 kB 11.6 MB/s eta 0:00:00


In [3]:
shutil.copytree('/content/drive/MyDrive/yolo_data', '/content/yolo_data')
shutil.copytree('/content/drive/MyDrive/folds', '/content/folds')

'/content/folds'

In [4]:
folds_directory = "/content/folds"

data_images = "/content/yolo_data/images"
data_labels = "/content/yolo_data/labels"
test_images = "/content/yolo_data/images/test"
test_labels = "/content/yolo_data/labels/test"

test_fold = 5

# Itera sobre os folds selecionados
for fold_number in range(1, 6):

    fold_images = os.path.join(folds_directory, f"fold{fold_number}_images")
    fold_labels = os.path.join(folds_directory, f"fold{fold_number}_labels")

    images_list = os.listdir(fold_images)
    labels_list = os.listdir(fold_labels)

    if fold_number == test_fold:
      for file_name in images_list:

        source_file = os.path.join(fold_images, file_name)

        destination_file = os.path.join(test_images, file_name)

        shutil.copy(source_file, destination_file)

      for file_name in labels_list:

        source_file = os.path.join(fold_labels, file_name)

        destination_file = os.path.join(test_labels, file_name)

        shutil.copy(source_file, destination_file)

    else:
      for file_name in images_list:

        source_file = os.path.join(fold_images, file_name)

        destination_file = os.path.join(data_images, file_name)

        shutil.copy(source_file, destination_file)

      for file_name in labels_list:

        source_file = os.path.join(fold_labels, file_name)

        destination_file = os.path.join(data_labels, file_name)

        shutil.copy(source_file, destination_file)

In [5]:
def dividir_train_val(pasta_imagens, pasta_textos, train_images, train_labels, val_images, val_labels, proporcao_val):

    arquivos_imagens = [arquivo for arquivo in os.listdir(pasta_imagens) if arquivo.endswith(".jpg")]

    arquivos_textos = [arquivo[:-4] + ".txt" for arquivo in arquivos_imagens if os.path.exists(os.path.join(pasta_textos, arquivo[:-4] + ".txt"))]

    quantidade_val = int(len(arquivos_imagens) * proporcao_val)

    arquivos_val = random.sample(arquivos_imagens, quantidade_val)

    for arquivo_imagem in arquivos_imagens:
        caminho_imagem = os.path.join(pasta_imagens, arquivo_imagem)
        caminho_texto = os.path.join(pasta_textos, arquivo_imagem[:-4] + ".txt")
        if arquivo_imagem in arquivos_val:
            shutil.move(caminho_imagem, os.path.join(val_images, arquivo_imagem))
            shutil.move(caminho_texto, os.path.join(val_labels, arquivo_imagem[:-4] + ".txt"))
        else:
            shutil.move(caminho_imagem, os.path.join(train_images, arquivo_imagem))
            shutil.move(caminho_texto, os.path.join(train_labels, arquivo_imagem[:-4] + ".txt"))

In [6]:
images = "/content/yolo_data/images"
label = "/content/yolo_data/labels"

train_img = "/content/yolo_data/images/train"
train_label = "/content/yolo_data/labels/train"

val_img = "/content/yolo_data/images/val"
val_label = "/content/yolo_data/labels/val"

dividir_train_val(images, label, train_img, train_label, val_img, val_label, 0.2)

In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.122 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [8]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/yolo_data/custom.yaml epochs=25 imgsz=640

100% 6.23M/6.23M [00:00<00:00, 22.1MB/s]
Ultralytics YOLOv8.0.122 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/yolo_data/custom.yaml, epochs=25, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, ker

In [9]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 agnostic_nms=True source=/content/yolo_data/images/test save_txt=true show_conf=false

Ultralytics YOLOv8.0.122 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients

image 1/43 /content/yolo_data/images/test/A2vid1_frame_0.jpg: 640x512 37 cars, 53.8ms
image 2/43 /content/yolo_data/images/test/A2vid1_frame_17.jpg: 640x512 37 cars, 6.0ms
image 3/43 /content/yolo_data/images/test/A2vid1_frame_3.jpg: 640x512 39 cars, 5.9ms
image 4/43 /content/yolo_data/images/test/A2vid1_frame_6.jpg: 640x512 37 cars, 5.9ms
image 5/43 /content/yolo_data/images/test/A2vid2_frame_10.jpg: 640x512 38 cars, 6.0ms
image 6/43 /content/yolo_data/images/test/A2vid2_frame_13.jpg: 640x512 39 cars, 5.9ms
image 7/43 /content/yolo_data/images/test/A2vid2_frame_14.jpg: 640x512 38 cars, 10.1ms
image 8/43 /content/yolo_data/images/test/A2vid2_frame_2.jpg: 640x512 39 cars, 8.9ms
image 9/43 /content/yolo_data/images/test/A2vid2_frame_23.jpg: 640x512 37 cars, 8.6ms
image 10/43 /content/yolo_data/images/test/A2vid2_frame_25.jpg: 640x512 

In [10]:
def contar_linhas_arquivo(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
    return len(linhas)

In [11]:
pasta_teste = '/content/yolo_data/labels/test'
pasta_predict = '/content/runs/detect/predict/labels'

porcentagens_acerto = []

for filename in os.listdir(pasta_teste):
    caminho_verdade = os.path.join(pasta_teste, filename)
    caminho_predict = os.path.join(pasta_predict, filename)

    if filename.endswith('.txt') and os.path.isfile(caminho_verdade):
        verdade = contar_linhas_arquivo(caminho_verdade)

    if filename.endswith('.txt') and os.path.isfile(caminho_predict):
        predict = contar_linhas_arquivo(caminho_predict)

    dif = verdade - predict
    nome_arquivo = os.path.splitext(os.path.basename(caminho_verdade))[0]

    print("Arquivo:", nome_arquivo)
    print("Diferença = ", abs(dif))

    porcentagem_acerto = 100 - (abs(dif) / max(verdade, predict)) * 100
    porcentagem_acerto = round(porcentagem_acerto, 2)
    porcentagens_acerto.append(porcentagem_acerto)

    print("Porcentagem de Acerto = ", porcentagem_acerto,"%")
    print("--------------------------------------------")

media_porcentagens = np.mean(porcentagens_acerto)
media_porcentagens = round(porcentagem_acerto, 2)
print("Média das Porcentagens de Acerto (CONF == 0.3) =", media_porcentagens,"%")

Arquivo: A3vid1_frame_19
Diferença =  4
Porcentagem de Acerto =  89.47 %
--------------------------------------------
Arquivo: A2vid2_frame_14
Diferença =  0
Porcentagem de Acerto =  100.0 %
--------------------------------------------
Arquivo: A3vid1_frame_36
Diferença =  10
Porcentagem de Acerto =  76.74 %
--------------------------------------------
Arquivo: A3vid4_frame_0
Diferença =  1
Porcentagem de Acerto =  97.37 %
--------------------------------------------
Arquivo: A2vid2_frame_57
Diferença =  4
Porcentagem de Acerto =  90.48 %
--------------------------------------------
Arquivo: A3vid1_frame_26
Diferença =  6
Porcentagem de Acerto =  84.62 %
--------------------------------------------
Arquivo: A2vid2_frame_10
Diferença =  0
Porcentagem de Acerto =  100.0 %
--------------------------------------------
Arquivo: A2vid1_frame_0
Diferença =  1
Porcentagem de Acerto =  97.37 %
--------------------------------------------
Arquivo: A2vid2_frame_50
Diferença =  1
Porcentagem de Ac

In [12]:
pasta_teste = '/content/yolo_data/labels/test'
pasta_predict = '/content/runs/detect/predict2/labels'

porcentagens_acerto = []

for filename in os.listdir(pasta_teste):
    caminho_verdade = os.path.join(pasta_teste, filename)
    caminho_predict = os.path.join(pasta_predict, filename)

    if filename.endswith('.txt') and os.path.isfile(caminho_verdade):
        verdade = contar_linhas_arquivo(caminho_verdade)

    if filename.endswith('.txt') and os.path.isfile(caminho_predict):
        predict = contar_linhas_arquivo(caminho_predict)

    dif = verdade - predict
    nome_arquivo = os.path.splitext(os.path.basename(caminho_verdade))[0]

    print("Arquivo:", nome_arquivo)
    print("Diferença = ", abs(dif))

    porcentagem_acerto = 100 - (abs(dif) / max(verdade, predict)) * 100
    porcentagem_acerto = round(porcentagem_acerto, 2)
    porcentagens_acerto.append(porcentagem_acerto)

    print("Porcentagem de Acerto = ", porcentagem_acerto, "%")
    print("--------------------------------------------")

media_porcentagens = np.mean(porcentagens_acerto)
print("Média das Porcentagens de Acerto (CONF == 0.4) =", media_porcentagens, "%")

Arquivo: A3vid1_frame_19
Diferença =  10
Porcentagem de Acerto =  77.27 %
--------------------------------------------
Arquivo: A2vid2_frame_14
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A3vid1_frame_36
Diferença =  11
Porcentagem de Acerto =  75.0 %
--------------------------------------------
Arquivo: A3vid4_frame_0
Diferença =  7
Porcentagem de Acerto =  84.09 %
--------------------------------------------
Arquivo: A2vid2_frame_57
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A3vid1_frame_26
Diferença =  11
Porcentagem de Acerto =  75.0 %
--------------------------------------------
Arquivo: A2vid2_frame_10
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A2vid1_frame_0
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A2vid2_frame_50
Diferença =  6
Porcentagem de Ac

In [13]:
pasta_teste = '/content/yolo_data/labels/test'
pasta_predict = '/content/runs/detect/predict3/labels'

porcentagens_acerto = []

for filename in os.listdir(pasta_teste):
    caminho_verdade = os.path.join(pasta_teste, filename)
    caminho_predict = os.path.join(pasta_predict, filename)

    if filename.endswith('.txt') and os.path.isfile(caminho_verdade):
        verdade = contar_linhas_arquivo(caminho_verdade)

    if filename.endswith('.txt') and os.path.isfile(caminho_predict):
        predict = contar_linhas_arquivo(caminho_predict)

    dif = verdade - predict
    nome_arquivo = os.path.splitext(os.path.basename(caminho_verdade))[0]

    print("Arquivo:", nome_arquivo)
    print("Diferença = ", abs(dif))

    porcentagem_acerto = 100 - (abs(dif) / max(verdade, predict)) * 100
    porcentagem_acerto = round(porcentagem_acerto, 2)
    porcentagens_acerto.append(porcentagem_acerto)

    print("Porcentagem de Acerto = ", porcentagem_acerto, "%")
    print("--------------------------------------------")

media_porcentagens = np.mean(porcentagens_acerto)
print("Média das Porcentagens de Acerto (CONF == 0.2) =", media_porcentagens, "%")

Arquivo: A3vid1_frame_19
Diferença =  10
Porcentagem de Acerto =  77.27 %
--------------------------------------------
Arquivo: A2vid2_frame_14
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A3vid1_frame_36
Diferença =  11
Porcentagem de Acerto =  75.0 %
--------------------------------------------
Arquivo: A3vid4_frame_0
Diferença =  7
Porcentagem de Acerto =  84.09 %
--------------------------------------------
Arquivo: A2vid2_frame_57
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A3vid1_frame_26
Diferença =  11
Porcentagem de Acerto =  75.0 %
--------------------------------------------
Arquivo: A2vid2_frame_10
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A2vid1_frame_0
Diferença =  6
Porcentagem de Acerto =  86.36 %
--------------------------------------------
Arquivo: A2vid2_frame_50
Diferença =  6
Porcentagem de Ac